In [1]:
from collections import defaultdict

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib import rcParams
import matplotlib.cm as cm
import matplotlib as mpl

from scipy import spatial
import timeit

In [203]:
X_train = pd.read_csv("../feature/X_train.csv")
y_train = pd.read_csv("../feature/y_train.csv")


In [204]:
X_train.head()

user_id             business_id  average_stars  \
0  VDh1vjzpNUJH6HfcjH8g7Q  WPCgtEG-bJt0cZtnM-x7yw           4.50   
1  3wzomxm2mM_VKuFvAol1LQ  WPCgtEG-bJt0cZtnM-x7yw           3.74   
2  3n3HEhr_yzPUzzV_JYkziw  WPCgtEG-bJt0cZtnM-x7yw           3.37   
3  ZCzUEzVQtT5meQctrdRD6w  WPCgtEG-bJt0cZtnM-x7yw           3.81   
4  c28lKPf7j0cSOZHa8kno3A  WPCgtEG-bJt0cZtnM-x7yw           3.61   

   compliment_cool  compliment_cute  compliment_funny  compliment_hot  \
0                0                1                 0               0   
1              290               81               290             259   
2                7                0                 7               4   
3                3                1                 3               2   
4               10                1                10               6   

   compliment_list  compliment_more  ...   attributes_WiFi  is_open  \
0                0                0  ...                 0        1   
1               26               59  ...                 0        1   
2                1                2  ...                 0        1   
3                1               19  ...                 0        1   
4                2                3  ...                 0        1   

   review_count_y  stars  state  cate_Restaurants  cate_Entertainment  \
0             128    4.0     17                 0                   0   
1             128    4.0     17                 0                   0   
2             128    4.0     17                 0                   0   
3             128    4.0     17                 0                   0   
4             128    4.0     17                 0                   0   

   cate_Travel  city  
0            0    13  
1            0    13  
2            0    13  
3            0    13  
4            0    13  

[5 rows x 59 columns]

In [205]:
print(X_train.shape, '\n')
print(X_train.columns)

(150232, 59) 

Index(['user_id', 'business_id', 'average_stars', 'compliment_cool',
       'compliment_cute', 'compliment_funny', 'compliment_hot',
       'compliment_list', 'compliment_more', 'compliment_note',
       'compliment_photos', 'compliment_plain', 'compliment_profile',
       'compliment_writer', 'cool', 'fans', 'funny', 'review_count_x',
       'useful', 'date_since', 'attributes_AcceptsInsurance',
       'attributes_AgesAllowed', 'attributes_Alcohol', 'attributes_BYOB',
       'attributes_BikeParking', 'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards', 'attributes_ByAppointmentOnly',
       'attributes_Caters', 'attributes_CoatCheck', 'attributes_Corkage',
       'attributes_DogsAllowed', 'attributes_DriveThru',
       'attributes_GoodForDancing', 'attributes_GoodForKids',
       'attributes_HappyHour', 'attributes_HasTV', 'attributes_NoiseLevel',
       'attributes_Open24Hours', 'attributes_OutdoorSeating',
       'attributes_Restaurants

In [206]:
pd.options.display.max_columns = X_train.shape[1]
X_train.describe()

average_stars  compliment_cool  compliment_cute  compliment_funny  \
count  150232.000000     150232.00000    150232.000000      150232.00000   
mean        3.754031        176.43745         9.472629         176.43745   
std         0.449233        659.02647        96.453304         659.02647   
min         1.000000          0.00000         0.000000           0.00000   
25%         3.550000          1.00000         0.000000           1.00000   
50%         3.770000         12.00000         0.000000          12.00000   
75%         4.000000         88.00000         2.000000          88.00000   
max         5.000000      12478.00000      2250.000000       12478.00000   

       compliment_hot  compliment_list  compliment_more  compliment_note  \
count   150232.000000    150232.000000    150232.000000    150232.000000   
mean       122.173998         5.812077        13.621592        75.948340   
std        662.018857        52.983797        70.911829       245.576159   
min          0.000000         0.000000         0.000000         0.000000   
25%          0.000000         0.000000         0.000000         1.000000   
50%          6.000000         0.000000         2.000000         9.000000   
75%         51.000000         1.000000         8.000000        41.000000   
max      19085.000000      1305.000000      1440.000000      4659.000000   

       compliment_photos  compliment_plain  compliment_profile  \
count      150232.000000     150232.000000       150232.000000   
mean           76.002137        185.429589           12.076681   
std           534.716551        644.988956          104.265938   
min             0.000000          0.000000            0.000000   
25%             0.000000          1.000000            0.000000   
50%             1.000000         11.000000            1.000000   
75%            12.000000         67.000000            3.000000   
max         10820.000000      13075.000000         2327.000000   

       compliment_writer           cool           fans          funny  \
count      150232.000000  150232.000000  150232.000000  150232.000000   
mean           68.419844     909.420556      49.614183     651.844268   
std           256.055421    4490.499134     110.410389    3545.488483   
min             0.000000       0.000000       0.000000       0.000000   
25%             0.000000       1.000000       2.000000       2.000000   
50%             7.000000      15.000000      12.000000      19.000000   
75%            37.000000     162.000000      43.000000     125.000000   
max          4352.000000   67566.000000    1593.000000   65804.000000   

       review_count_x         useful     date_since  \
count   150232.000000  150232.000000  150232.000000   
mean       426.692835    1019.347256    2010.612593   
std        712.941849    4471.081979       2.125944   
min          0.000000       0.000000    2005.000000   
25%         66.000000       9.000000    2009.000000   
50%        201.000000      64.000000    2011.000000   
75%        497.000000     376.000000    2012.000000   
max      11942.000000   68448.000000    2016.000000   

       attributes_AcceptsInsurance  attributes_AgesAllowed  \
count                150232.000000           150232.000000   
mean                     -0.992565                0.009918   
std                       0.109299                0.099094   
min                      -1.000000                0.000000   
25%                      -1.000000                0.000000   
50%                      -1.000000                0.000000   
75%                      -1.000000                0.000000   
max                       1.000000                1.000000   

       attributes_Alcohol  attributes_BYOB  attributes_BikeParking  \
count       150232.000000         150232.0           150232.000000   
mean             0.886955              3.0                0.592810   
std              0.914141              0.0                0.679261   
min              0.000000              3.0       

In [207]:
biz_cols = [
       'business_id', 
       'is_open',
       'stars', ## different?
       'review_count_y', ## different
    
       'city',
       'state',
       
       'cate_Restaurants',
       'cate_Entertainment',
       'cate_Travel',
    
       'attributes_AcceptsInsurance',
       'attributes_AgesAllowed', 
       'attributes_Alcohol',
       'attributes_BYOB',
       'attributes_BikeParking', 
       'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards', 
       'attributes_ByAppointmentOnly', 
       'attributes_Caters',
       'attributes_CoatCheck', 
       'attributes_Corkage',
       'attributes_DogsAllowed',
       'attributes_DriveThru', 
       'attributes_GoodForDancing',
       'attributes_GoodForKids', 
       'attributes_HappyHour',
       'attributes_HasTV',  
       'attributes_NoiseLevel',
       'attributes_Open24Hours', 
       'attributes_OutdoorSeating',
       'attributes_RestaurantsAttire', 
       'attributes_RestaurantsCounterService',
       'attributes_RestaurantsDelivery',
       'attributes_RestaurantsGoodForGroups',
       'attributes_RestaurantsPriceRange2',
       'attributes_RestaurantsReservations',
       'attributes_RestaurantsTableService', 
       'attributes_RestaurantsTakeOut',
       'attributes_Smoking', 
       'attributes_WheelchairAccessible',
       'attributes_WiFi'
       ]



In [208]:
# extract business info
business = X_train[biz_cols]

print(business.shape)
# business.head()

# drop duplicates
business = business.drop_duplicates()

print(business.shape)
# business.head()

pd.options.display.max_columns = business.shape[1]
business.describe()

(150232, 40)
(10572, 40)


is_open         stars  review_count_y          city    state  \
count  10572.000000  10572.000000    10572.000000  10572.000000  10572.0   
mean       0.880344      3.743331      276.939936      5.055713     17.0   
std        0.324574      0.625074      341.504014      5.767515      0.0   
min        0.000000      1.000000      100.000000      0.000000     17.0   
25%        1.000000      3.500000      129.000000      0.000000     17.0   
50%        1.000000      4.000000      178.000000      3.000000     17.0   
75%        1.000000      4.000000      292.000000      8.000000     17.0   
max        1.000000      5.000000     7968.000000     17.000000     17.0   

       cate_Restaurants  cate_Entertainment  cate_Travel  \
count      10572.000000        10572.000000  10572.00000   
mean           0.739217            0.054389      0.03689   
std            0.439083            0.226794      0.18850   
min            0.000000            0.000000      0.00000   
25%            0.000000            0.000000      0.00000   
50%            1.000000            0.000000      0.00000   
75%            1.000000            0.000000      0.00000   
max            1.000000            1.000000      1.00000   

       attributes_AcceptsInsurance  attributes_AgesAllowed  \
count                 10572.000000            10572.000000   
mean                     -0.980420                0.005581   
std                       0.176938                0.074499   
min                      -1.000000                0.000000   
25%                      -1.000000                0.000000   
50%                      -1.000000                0.000000   
75%                      -1.000000                0.000000   
max                       1.000000                1.000000   

       attributes_Alcohol  attributes_BYOB  attributes_BikeParking  \
count        10572.000000          10572.0            10572.000000   
mean             1.025918              3.0                0.601684   
std              0.926027              0.0                0.701478   
min              0.000000              3.0               -1.000000   
25%              0.000000              3.0                0.000000   
50%              1.000000              3.0                1.000000   
75%              2.000000              3.0                1.000000   
max              2.000000              3.0                1.000000   

       attributes_BusinessAcceptsBitcoin  \
count                       10572.000000   
mean                           -0.862278   
std                             0.353031   
min                            -1.000000   
25%                            -1.000000   
50%                            -1.000000   
75%                            -1.000000   
max                             1.000000   

       attributes_BusinessAcceptsCreditCards  attributes_ByAppointmentOnly  \
count                           10572.000000                  10572.000000   
mean                                0.893114                     -0.861521   
std                                 0.420731                      0.420061   
min                                -1.000000                     -1.000000   
25%                                 1.000000                     -1.000000   
50%                                 1.000000                     -1.000000   
75%                                 1.000000                     -1.000000   
max                                 1.000000                      1.000000   

       attributes_Caters  attributes_CoatCheck  attributes_Corkage  \
count       10572.000000          10572.000000        10572.000000   
mean            0.179815             -0.806092           -0.951476   
std             0.781561              0.442994            0.257717   
min            -1.000000             -1.000000           -1.000000   
25%             0.000000             -1.000000           -1.000000   
50%             0.000000             -1.000000           -1.000000   
75%   

In [209]:
biz_attrs = [
       'attributes_AcceptsInsurance',
       'attributes_AgesAllowed', 
       'attributes_Alcohol',
       'attributes_BYOB',
       'attributes_BikeParking', 
       'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards', 
       'attributes_ByAppointmentOnly', 
       'attributes_Caters',
       'attributes_CoatCheck', 
       'attributes_Corkage',
       'attributes_DogsAllowed',
       'attributes_DriveThru', 
       'attributes_GoodForDancing',
       'attributes_GoodForKids', 
       'attributes_HappyHour',
       'attributes_HasTV',  
       'attributes_NoiseLevel',
       'attributes_Open24Hours', 
       'attributes_OutdoorSeating',
       'attributes_RestaurantsAttire', 
       'attributes_RestaurantsCounterService',
       'attributes_RestaurantsDelivery',
       'attributes_RestaurantsGoodForGroups',
       'attributes_RestaurantsPriceRange2',
       'attributes_RestaurantsReservations',
       'attributes_RestaurantsTableService', 
       'attributes_RestaurantsTakeOut',
       'attributes_Smoking', 
       'attributes_WheelchairAccessible',
       'attributes_WiFi'
       ];
      
n_biz_attrs = len(biz_attrs)
print(n_biz_attrs)

31


In [210]:
# (minmax) normalize values of attributes 
business[biz_attrs] = business[biz_attrs].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)) if np.max(x) != np.min(x) else 0.0)

print(business.shape)
# business.head()

pd.options.display.max_columns = business.shape[1]
business.describe()

(10572, 40)


is_open         stars  review_count_y          city    state  \
count  10572.000000  10572.000000    10572.000000  10572.000000  10572.0   
mean       0.880344      3.743331      276.939936      5.055713     17.0   
std        0.324574      0.625074      341.504014      5.767515      0.0   
min        0.000000      1.000000      100.000000      0.000000     17.0   
25%        1.000000      3.500000      129.000000      0.000000     17.0   
50%        1.000000      4.000000      178.000000      3.000000     17.0   
75%        1.000000      4.000000      292.000000      8.000000     17.0   
max        1.000000      5.000000     7968.000000     17.000000     17.0   

       cate_Restaurants  cate_Entertainment  cate_Travel  \
count      10572.000000        10572.000000  10572.00000   
mean           0.739217            0.054389      0.03689   
std            0.439083            0.226794      0.18850   
min            0.000000            0.000000      0.00000   
25%            0.000000            0.000000      0.00000   
50%            1.000000            0.000000      0.00000   
75%            1.000000            0.000000      0.00000   
max            1.000000            1.000000      1.00000   

       attributes_AcceptsInsurance  attributes_AgesAllowed  \
count                 10572.000000            10572.000000   
mean                      0.009790                0.005581   
std                       0.088469                0.074499   
min                       0.000000                0.000000   
25%                       0.000000                0.000000   
50%                       0.000000                0.000000   
75%                       0.000000                0.000000   
max                       1.000000                1.000000   

       attributes_Alcohol  attributes_BYOB  attributes_BikeParking  \
count        10572.000000          10572.0            10572.000000   
mean             0.512959              0.0                0.800842   
std              0.463013              0.0                0.350739   
min              0.000000              0.0                0.000000   
25%              0.000000              0.0                0.500000   
50%              0.500000              0.0                1.000000   
75%              1.000000              0.0                1.000000   
max              1.000000              0.0                1.000000   

       attributes_BusinessAcceptsBitcoin  \
count                       10572.000000   
mean                            0.068861   
std                             0.176516   
min                             0.000000   
25%                             0.000000   
50%                             0.000000   
75%                             0.000000   
max                             1.000000   

       attributes_BusinessAcceptsCreditCards  attributes_ByAppointmentOnly  \
count                           10572.000000                  10572.000000   
mean                                0.946557                      0.069240   
std                                 0.210365                      0.210031   
min                                 0.000000                      0.000000   
25%                                 1.000000                      0.000000   
50%                                 1.000000                      0.000000   
75%                                 1.000000                      0.000000   
max                                 1.000000                      1.000000   

       attributes_Caters  attributes_CoatCheck  attributes_Corkage  \
count       10572.000000          10572.000000        10572.000000   
mean            0.589907              0.096954            0.024262   
std             0.390781              0.221497            0.128859   
min             0.000000              0.000000            0.000000   
25%             0.500000              0.000000            0.000000   
50%             0.500000              0.000000            0.000000   
75%   

In [211]:
business.index

Int64Index([     0,     11,     18,     78,    182,    187,    216,    233,
               249,    300,
            ...
            150222, 150223, 150224, 150225, 150226, 150227, 150228, 150229,
            150230, 150231],
           dtype='int64', length=10572)

In [212]:
# reset the indices of business
# for convenience of indexing in biz_sim_mat
business = business.reset_index(drop=True)
business.index

RangeIndex(start=0, stop=10572, step=1)

In [213]:
# print(biz_attrs[0], '\n')
# print(business.iloc[0], '\n')

# print(business.iloc[0][biz_attrs[0]], '\n')
# print(business.iloc[0][biz_attrs])

# biz = business.iloc[i]
# l = biz[biz_attrs].values.tolist()
# print(1 - spatial.distance.cosine(l, l))

In [254]:
biz_sim_mat = np.identity(business.shape[0])
print(biz_sim_mat.shape)

(10572, 10572)


In [255]:
# # too slow...
# # must try to accelerate it

# start = timeit.default_timer()

# # for i in range(business.shape[0]-1):  
# for i in range(1-1):      
#     biz_1 = business.iloc[i]
#     l_1 = biz_1[biz_attrs].values.tolist()
    
#     for j in range(i+1, 1):
#     # for j in range(i+1, business.shape[0]):
#         biz_2 = business.iloc[j]
#         l_2 = biz_2[biz_attrs].values.tolist()
    
#         # any other similarity measure?
#         # how to reflect the influence of review numbers for a business
#         sim = 1 - spatial.distance.cosine(l_1, l_2)
        
#         biz_sim_mat[i,j] = sim
#         # biz_sim_mat[j,i] = sim


# stop = timeit.default_timer()
# print('Time: ', stop - start) 
# print('biz_sim_mat: \n', biz_sim_mat)

Time:  0.0001047550467774272
biz_sim_mat: 
 [[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [258]:
# alternative way: 
# just calculate similarity of 2 businesses when needed
# instead of computing & then stroing similarities of all possible pairs in a matrix
def calculate_similarity(biz_1_index, biz_2_index):
    # thanks to statement "business = business.reset_index(drop=True)" (???)
    # usually default (consecutive) indices works
    biz_1 = business.iloc[biz_1_index]
    l_1 = biz_1[biz_attrs].values.tolist()
    
    biz_2 = business.iloc[biz_2_index]
    l_2 = biz_2[biz_attrs].values.tolist()
    
    # any other similarity measure?
    # how to reflect the influence of review numbers for a business
    sim = 1 - spatial.distance.cosine(l_1, l_2)
    return sim

In [259]:
# # print('unweighted business:', X_train['stars'].mean())
# print('unweighted business:', business['stars'].mean())

# print('unweighted reviews:', y_train['stars'].mean())

In [260]:
user_cols = [
    'user_id',
    'average_stars', 
    'compliment_cool', 
    'compliment_cute',
    'compliment_funny', 
    'compliment_hot', 
    'compliment_list',
    'compliment_more', 
    'compliment_note', 
    'compliment_photos',
    'compliment_plain', 
    'compliment_profile', 
    'compliment_writer', 
    'cool',
    'fans', 
    'funny', 
    'review_count_x', ##different
    'useful',
    'date_since'
]


In [261]:
user = X_train[user_cols]

print(user.shape)
# user.head()

# drop duplicates
user = user.drop_duplicates()

print(user.shape)
# user.head()

pd.options.display.max_columns = user.shape[1]
user.describe()

(150232, 19)
(16556, 19)


average_stars  compliment_cool  compliment_cute  compliment_funny  \
count   16556.000000     16556.000000     16556.000000      16556.000000   
mean        3.762765        22.581058         1.277301         22.581058   
std         0.749822       260.605613        24.703059        260.605613   
min         1.000000         0.000000         0.000000          0.000000   
25%         3.400000         0.000000         0.000000          0.000000   
50%         3.830000         0.000000         0.000000          0.000000   
75%         4.210000         1.000000         0.000000          1.000000   
max         5.000000     12478.000000      2250.000000      12478.000000   

       compliment_hot  compliment_list  compliment_more  compliment_note  \
count    16556.000000     16556.000000     16556.000000     16556.000000   
mean        16.826045         0.562938         1.912237         9.607272   
std        244.200696        12.114683        20.173730        89.876249   
min          0.000000         0.000000         0.000000         0.000000   
25%          0.000000         0.000000         0.000000         0.000000   
50%          0.000000         0.000000         0.000000         0.000000   
75%          1.000000         0.000000         0.000000         2.000000   
max      19085.000000      1305.000000      1440.000000      4659.000000   

       compliment_photos  compliment_plain  compliment_profile  \
count       16556.000000      16556.000000        16556.000000   
mean            8.552911         21.805327            1.428666   
std           170.721359        249.041592           27.400352   
min             0.000000          0.000000            0.000000   
25%             0.000000          0.000000            0.000000   
50%             0.000000          0.000000            0.000000   
75%             0.000000          2.000000            0.000000   
max         10820.000000      13075.000000         2327.000000   

       compliment_writer          cool          fans         funny  \
count       16556.000000  16556.000000  16556.000000  16556.000000   
mean            8.163385    122.397560      7.228558     91.585165   
std            88.040084   1492.021745     35.167493   1146.844961   
min             0.000000      0.000000      0.000000      0.000000   
25%             0.000000      0.000000      0.000000      0.000000   
50%             0.000000      0.000000      0.000000      1.000000   
75%             1.000000      3.000000      3.000000      5.000000   
max          4352.000000  67566.000000   1593.000000  65804.000000   

       review_count_x       useful    date_since  
count    16556.000000  16556.00000  16556.000000  
mean        82.601051    161.80720   2011.663566  
std        218.914853   1600.18848      2.184983  
min          0.000000      0.00000   2005.000000  
25%          8.000000      0.00000   2010.000000  
50%         21.000000      3.00000   2012.000000  
75%         69.000000     21.00000   2013.000000  
max      11942.000000  68448.00000   2016.000000

In [262]:
Y_bar      = y_train['stars'].mean()
Y_user_bar = user['average_stars'].mean()
Y_biz_bar  = business['stars'].mean()
print(Y_bar, Y_user_bar, Y_biz_bar)

3.7641447893924065 3.7627645566561974 3.7433314415437002


In [297]:
def pred_baseline(user_id, business_id):
# credit to henryachen 
# https://gist.github.com/henryachen/a5fb0281c1715e83aaef
    pred = Y_bar
    
    u = user[user['user_id'] == user_id]
    if u.shape[0] == 0:
        pred += 0
    else:
        pred += u['average_stars'].values[0] - Y_user_bar
    
    m = business[business['business_id'] == business_id]
    if m.shape[0] == 0:
        pred += 0
    else:
        pred += m['stars'].values[0]         - Y_biz_bar
        
    return pred

In [298]:
# print(pred_baseline(X_train.loc[0, 'user_id'], X_train.loc[0, 'business_id']))
# print(y_train.loc[0][0])

In [299]:
# x = X_test.iloc[0]
# print(x['user_id'])

# print(X_train[X_train['user_id'] == 'VDh1vjzpNUJH6HfcjH8g7Q'])
# print('\n')
# print(X_train.index[X_train['user_id'] == 'VDh1vjzpNUJH6HfcjH8g7Q']) # index

# u = X_train[X_train['user_id'] == 'VDh1vjzpNUJH6HfcjH8g7Q']
# print(u)
# ids = u["business_id"].values.tolist()
# idxs = u.index.values.tolist()
# print(ids)
# print(idxs)

In [300]:
def pred_more(user_id, business_id):
# credit to henryachen 
# https://gist.github.com/henryachen/a5fb0281c1715e83aaef
    pred_base = pred_baseline(user_id, business_id)
    
    q = X_train[X_train['user_id'] == user_id] # query X_train, not user
    # note the differences
    biz_ids          = q['business_id'].values.tolist()
    user_biz_indices = q.index.values.tolist() # review index in y_train, not biz index in business
    sz = len(biz_ids)
    
    business_index_TEMP = business.index[business["business_id"] == business_id] # might be not found
    
    if sz == 0 or business_index_TEMP.shape[0] == 0: # two conditions
        return pred_base
    else:
        business_index = business_index_TEMP.values[0]
        # weighted average
        den, num = 0, 0
        for i in range(sz):
            #
            biz_id = biz_ids[i]
            biz_index  = business.index[business["business_id"] == biz_id].values[0]
            user_biz_index = user_biz_indices[i]
            Y_user_biz = y_train.loc[user_biz_index, 'stars']
            #
            # sim = biz_sim_mat[business_index, biz_index]
            sim = calculate_similarity(business_index, biz_index)
            pb = pred_baseline(user_id, biz_id)
            den += sim
            num += sim * (Y_user_biz - pb)
            
        return pred_base + num / den

In [301]:
print('Example 0')
print('real:     ', y_train.loc[0, 'stars'])
print('baseline: ', pred_baseline(X_train.loc[0, 'user_id'], X_train.loc[0, 'business_id']))
print('more:     ', pred_more(X_train.loc[0, 'user_id'], X_train.loc[0, 'business_id']))

Example 0
real:      4.0
baseline:  4.758048791192509
more:      3.2766903593842125


In [294]:
X_test = pd.read_csv("../data/test_queries.csv")

In [276]:
# runs about 2 hours (for 50,000 samples)...
# seems very slow...
# need better methods...
ls = []
for i in range(X_test.shape[0]):
#     if i % 1000 == 0:
#         print(i) 
    x = X_test.iloc[i]
    p = pred_more(x['user_id'], x['business_id'])
    ls.append(p)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

/Users/phoenix/anaconda3/envs/.env/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


30400
30500
30600
30700
30800
30900
31000
31100
31200
31300
31400
31500
31600
31700
31800
31900
32000
32100
32200
32300
32400
32500
32600
32700
32800
32900
33000
33100
33200
33300
33400
33500
33600
33700
33800
33900
34000
34100
34200
34300
34400
34500
34600
34700
34800
34900
35000
35100
35200
35300
35400
35500
35600
35700
35800
35900
36000
36100
36200
36300
36400
36500
36600
36700
36800
36900
37000
37100
37200
37300
37400
37500
37600
37700
37800
37900
38000
38100
38200
38300
38400
38500
38600
38700
38800
38900
39000
39100
39200
39300
39400
39500
39600
39700
39800
39900
40000
40100
40200
40300
40400
40500
40600
40700
40800
40900
41000
41100
41200
41300
41400
41500
41600
41700
41800
41900
42000
42100
42200
42300
42400
42500
42600
42700
42800
42900
43000
43100
43200
43300
43400
43500
43600
43700
43800
43900
44000
44100
44200
44300
44400
44500
44600
44700
44800
44900
45000
45100
45200
45300
45400
45500
45600
45700
45800
45900
46000
46100
46200
46300
46400
46500
46600
46700
46800
46900
4700

In [286]:
# a bug...
# just manually fill in 2.5 to the entry of this case in output .csv file lol
# will fix it later
x = X_test.iloc[30369]
u = x['user_id']
m = x['business_id']
print(u, m)

q = X_train[X_train['user_id'] == u]
print(business.index[business["business_id"] == m].shape[0])

pred_more(u, m)

U-3Hwe0oEwQV5XxqiaypbA QvltB7RjVOVRBl685azJ5g
1


/Users/phoenix/anaconda3/envs/.env/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars


nan

In [287]:
df = pd.DataFrame(np.array(ls).reshape(-1,1), columns = {'stars'})

In [288]:
df.to_csv("../feature/X_test_output.csv", index_label='index')